## preprocessing data(normalization and standardization)
- In terms of scaling values, enural networks tend to prefer normalization 
- if youre not sure on which to use , you could try both and see which preforms better 

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import sklearn 

# importing the data 
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
# first we need to use sklearn to transform out data 
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
# now lets create a column transformaer to transform the column 
ct = make_column_transformer(
    (MinMaxScaler(), ["age", "bmi", "children"]), 
    (OneHotEncoder(handle_unknown = "ignore"), ["sex", "smoker", "region"]) # here it is going to one hit encode the values 
    ) # takes in StandardScalar or MinMaxScalar and OneHotEncoder 
# we want to tunr all the values in these columns into vlause betwen 0 and on ebecasue they are numerical vlaes, they are not objects of
X = insurance.drop("charges" , axis = 1)
y = insurance ["charges"]
# now we want to the model to learn the data and need to split data 
X_train,X_test, y_train, y_test =train_test_split(X,y, test_size = 0.2, random_state = 42)
# fitting the column transformer to our training data 
ct.fit(X_train) 
# transform training and test data with automatic with normalization from the training data 
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [11]:
X_train_normal


array([[0.60869565, 0.10734463, 0.4       , ..., 1.        , 0.        ,
        0.        ],
       [0.63043478, 0.22491256, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.73913043, 0.23944041, 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.86956522, 0.24791499, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.41304348, 0.85122411, 0.4       , ..., 0.        , 0.        ,
        1.        ],
       [0.80434783, 0.37503363, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [13]:
X_train.shape, X_train_normal.shape

((1070, 6), (1070, 11))

In [16]:
# now lets fit a neural model network on the data
tf.random.set_seed(42)

normal_insurance_model3 = tf.keras.Sequential([
    tf.keras.layers.Dense(100), 
    tf.keras.layers.Dense(10), 
    tf.keras.layers.Dense(1)
])
normal_insurance_model3.compile(loss = tf.keras.losses.mae, 
                                optimizer = tf.keras.optimizers.Adam(),
                                metrics = ["mae"])
with tf.device('/cpu:0'): history2 = normal_insurance_model3.fit(X_train_normal, y_train, epochs=100, verbose=1) # we need to use the with cup or the kernal will crash



Epoch 1/100


2022-07-04 22:53:37.497991: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


34/34 [==============================] - 0s 2ms/step - loss: 13342.6494 - mae: 13342.6494
Epoch 2/100
25/34 [=====================>........] - ETA: 0s - loss: 13255.9375 - mae: 13255.9375

2022-07-04 22:53:37.777774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


34/34 [==============================] - 0s 2ms/step - loss: 13333.4795 - mae: 13333.4795
Epoch 3/100
34/34 [==============================] - 0s 2ms/step - loss: 13312.0234 - mae: 13312.0234
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 13267.7930 - mae: 13267.7930
Epoch 5/100
34/34 [==============================] - 0s 3ms/step - loss: 13189.5850 - mae: 13189.5850
Epoch 6/100
34/34 [==============================] - 0s 2ms/step - loss: 13066.4502 - mae: 13066.4502
Epoch 7/100
34/34 [==============================] - 0s 2ms/step - loss: 12888.1953 - mae: 12888.1953
Epoch 8/100
34/34 [==============================] - 0s 2ms/step - loss: 12644.6523 - mae: 12644.6523
Epoch 9/100
34/34 [==============================] - 0s 3ms/step - loss: 12325.5469 - mae: 12325.5469
Epoch 10/100
34/34 [==============================] - 0s 2ms/step - loss: 11925.9658 - mae: 11925.9658
Epoch 11/100
34/34 [==============================] - 0s 2ms/step - loss: 11454.3350 - mae: 1

In [17]:
# now we want to evlauate our model on the normalized data 
normal_insurance_model3.evaluate(X_test_normal, y_test)

2022-07-04 22:55:01.814301: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 7ms/step - loss: 3438.7844 - mae: 3438.7844


[3438.784423828125, 3438.784423828125]

In [18]:
#if we compare this to the output from the non normalised data ther eis a reduction of 30% in MAE
# 9/9 [==============================] - 0s 5ms/step - loss: 4924.4956 - mae: 4924.4956
#this was the output from our model 2 with on normalization
#done 